## How to implement an N-port device in Python to be used in PAN

In [ ]:
import pypan.ui as pan
import numpy as np
from scipy.optimize import fminbound
import matplotlib.pyplot as plt
%matplotlib inline

The purpose of this notebook is to show how to implement an n-port device in Python to be used in PAN.

### The netlist

We start by defining the netlist we will use: the <tt>PYRESISTOR</tt> model defines an n-port that will implement a linear resistor, analogous to the <tt>resistor</tt> device already present in PAN.

The <tt>setup</tt> and <tt>evaluate</tt> parameters of the <tt>nport</tt> device specify the names of the Python functions that will be called upon initialization (in this case <tt>pyres_setup</tt>) and during simulation (in this case <tt>pyres_eval</tt>). 

In [ ]:
R0 = 5
netlist = """\
ground electrical gnd

E1  x   gnd  vsource     dc=10
R1  x   y    resistor    r={}
I1  y   gnd  PYRESISTOR  resistance={}

model PYRESISTOR nport macro=yes setup="pyres_setup" evaluate="pyres_eval"
""".format(R0, 2*R0)

#### Write the above netlist to file

In [ ]:
netlist_file = 'nport.pan'
with open(netlist_file, 'w') as fid:
    fid.write(netlist)

### Initialization of the n-port

This function is called once upon initialization of the n-port and must return a dictionary, where each (key,value) pair represents the name and default value of a parameter of the n-port, in this case the resistance of the linear resistor.

These default values will be passed by PAN to the Python evaluation function if no parameter is passed to the specific instance of the model (i.e., if the netlist line

<tt>I1  y   gnd  PYRESISTOR  resistance=R</tt>

were replaced by

<tt>I1  y   gnd  PYRESISTOR</tt>

then the <tt>resistance</tt> parameter would automatically be set to the default value specified by <tt>pyres_setup</tt>.

We use a lambda function because the setup function must be callable, i.e. a constant dictionary would not be sufficient in this case.

In [ ]:
pyres_setup = lambda: {'resistance': 1}

### Evaluation of the n-port

This function defines the characteristic of the n-port.

It takes the following arguments:

1. <tt>n</tt>: a scalar indicating the number of ports of the n-port.
1. <tt>V</tt>: a NumPy array containing the port voltages.
1. <tt>I</tt>: a NumPy array containing the port currents.
1. <tt>time</tt>: a scalar containing the time instant at which the function is evaluated.
1. <tt>kwargs</tt>: a series of (key,value) pairs representing the parameters of the n-port, as specified by the setup function.

It must return the following values:

1. <tt>f</tt>: an <tt>n</tt>-by-1 array containing the values of the implicit algebraic equation(s) f(V,I) = 0 describing the n-port.
1. <tt>C</tt>: an <tt>n</tt>-by-<tt>n</tt> sensitivity matrix containing the derivatives of <tt>f</tt> with respect to the voltages.
1. <tt>R</tt>: an <tt>n</tt>-by-<tt>n</tt> sensitivity matrix containing the derivatives of <tt>f</tt> with respect to the currents.

In [ ]:
def pyres_eval(n, V, I, time, **kwargs):
    r = kwargs['resistance']
    f = np.array([V[0] - r * I[0]])
    C = np.array([1.0], ndmin=2)
    R = np.array([-r], ndmin=2)
    return f, C, R

### Load the netlist with the description of the circuit
No analysis is performed at this point, since none are defined in the netlist.

In [ ]:
ok,libs = pan.load_netlist(netlist_file)
if not ok:
    print('load_netlist failed.')

### Define a cost function for the maximization of the power absorbed by the PYRESISTOR

In [ ]:
def cost(R):
    cost_id = str(int(np.random.uniform() * 1e6))
    pan.alter('Al_' + cost_id, 'resistance', R, libs, instance='I1', invalidate='no', annotate=4)
    V = pan.DC('Dc_' + cost_id, ['y'], libs, print='yes')
    return -V[0][0] ** 2 / R

### Optimize the resistance value
The n-port modelling the linear resistor is in series with the <tt>R1</tt> resistor: therefore, it will absorb maximum power when its resistance is the same as that of <tt>R1</tt>.

In [ ]:
Ropt, _, _, n_eval = fminbound(cost, 0, 1000, full_output=True)

### Print the results

In [ ]:
print('   Optimal value of resistance: {:.2f} Ohm.'.format(Ropt, 2*R0))
print('Number of function evaluations: {}.'.format(n_eval))

We can easily verify that the obtained value of resistance is the correct one by performing a DC sweep analysis and computing the value of absorbed power for each value of resistance:

In [ ]:
R = np.linspace(R0 / 10, 10 * R0, 100)
V = pan.DC('Dcsweep', ['y'], libs, start=R[0], stop=R[-1], step=np.diff(R)[0], instance='I1', param='resistance')

### Plot the results

In [ ]:
fig,ax = plt.subplots(1, 1, figsize=(6,4))
ax.plot(R, V[0]**2 / R, 'k')
ax.plot(Ropt + np.zeros(2), plt.ylim(), 'r--', label='Optimal value')
ax.set_xlabel('Resistance [Ω]')
ax.set_ylabel('Power [W]')
ax.legend(loc='upper right')
ax.axis('tight')
fig.tight_layout()
for pos in 'top','right':
    ax.spines[pos].set_visible(False)